## Starting a Coroutine

In [2]:
import asyncio

async def coroutine():
    print('in coroutine')
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(coroutine())
finally:
    event_loop.close()

in coroutine


## Returning Values from Coroutines

In [3]:
import asyncio
async def coroutine():
    print('in coroutine')
    return 1234
event_loop = asyncio.get_event_loop()
try:
    result = event_loop.run_until_complete(coroutine())
    print(result)
finally:
    event_loop.close()

in coroutine
1234


## Chaining Coroutines

In [1]:
import asyncio
async def outter():
    result1 = await phase1()
    result2 = await phase2(result1)
    return result1, result2
    
async def phase1():
    return 'hahha'

async def phase2(arg):
    return arg+'recived'

event_loop = asyncio.get_event_loop()
try:
    result = event_loop.run_until_complete(outter())
    print(result)
finally:
    event_loop.close()

('hahha', 'hahharecived')


## Scheduling a Callback “Soon”

In [1]:
import asyncio
import functools
def callback(arg, *, kwargs='default'):
    print(f"callback with {arg} and {kwargs}")
    
async def main(loop):
    print('registering callbacks')
    loop.call_soon(callback, 1)
    wrapped = functools.partial(callback, kwargs='not_default')
    loop.call_soon(wrapped, 2)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

registering callbacks
callback with 1 and default
callback with 2 and not_default


## Scheduling a Callback with a Delay

In [1]:
import asyncio
def callback(n):
    print(f"callback with {n} ")
    
async def main(loop):
    print('registering callbacks')
    loop.call_later(0.2,callback, 1)
    loop.call_later(0.1,callback, 2)
    loop.call_soon(callback, 3)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

registering callbacks
callback with 3 
callback with 2 
callback with 1 


## Scheduling a Callback for a Specific Time

In [1]:
import asyncio
import time
def callback(n, loop):
    print(f"callback with {n} at {loop.time()} ")
    
async def main(loop):
    now = loop.time()
    print('clock time ', time.time())
    print('loop time ', now)
    loop.call_at(0.2+now,callback, 1, loop)
    loop.call_at(0.1+now,callback, 2, loop)
    loop.call_soon(callback, 3, loop)
    await asyncio.sleep(1)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

clock time  1538278157.53533
loop time  149226.490097989
callback with 3 at 149226.490335996 
callback with 2 at 149226.592940694 
callback with 1 at 149226.693366884 


## Waiting for a Future

In [1]:
import asyncio

def mark_done(future, result):
    print(f"setting future result for {result!r}")
    future.set_result(result)
    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    print('scheduling mark_done')
    event_loop.call_soon(mark_done, all_done, 'the result111')
    result = event_loop.run_until_complete(all_done)
    print('return result',result)
finally:
    event_loop.close()
print('final result ',all_done.result())

scheduling mark_done
setting future result for 'the result111'
return result the result111
final result  the result111


In [1]:
# future with await
import asyncio

def mark_done(future, result):
    print(f"setting future result for {result!r}")
    future.set_result(result)
    
async def main(loop):
    all_done = asyncio.Future()
    print('scheduling mark_done')
    loop.call_soon(mark_done, all_done, 'the result222')
    result =  await all_done
    print('return result',result)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

scheduling mark_done
setting future result for 'the result222'
return result the result222


## Future Callbacks

In [1]:
import asyncio
import functools

def callback(future, n):
    print(f"{n}: future done: {future.result()}")
    
async def reg_callbacks(all_done):
    all_done.add_done_callback(functools.partial(callback, n=1))
    all_done.add_done_callback(functools.partial(callback, n=3))
    
async def main(all_done):
    await reg_callbacks(all_done)
    all_done.set_result('sdasdsa')
    
event_loop = asyncio.get_event_loop()
try:
    all_done = asyncio.Future()
    event_loop.run_until_complete(main(all_done))
finally:
    event_loop.close()

1: future done: sdasdsa
3: future done: sdasdsa


## Starting a Task

In [1]:
import asyncio
    
async def task_func():
    print('exe task')
    return 'task_result'
    
async def main(loop):
    print('create task')
    task = loop.create_task(task_func())
    return_value = await task
    print(return_value)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

create task
exe task
task_result


## Canceling a Task

In [1]:
import asyncio
    
async def task_func():
    print('exe task')
    return 'task_result'
    
async def main(loop):
    print('create task')
    task = loop.create_task(task_func())
    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print('canceled task')
    else:
        print(task.result())
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

create task
canceled task


In [1]:
import asyncio
    
async def task_func():
    print('in task func, sleeping')
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError:
        print('task_func cancelled',3)
        raise
    return 'task_result'

def task_cancel(t):
    print('in task_cancel')
    t.cancel()
    print('task_cancel',2)
    
async def main(loop):
    print('create task')
    task = loop.create_task(task_func())
    loop.call_soon(task_cancel, task)
    try:
        await task
    except asyncio.CancelledError:
        print('main also cancel',4)
    else:
        print(task.result())
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

create task
in task func, sleeping
in task_cancel
task_cancel 2
task_func cancelled 3
main also cancel 4


## Creating Tasks from Coroutines

In [1]:
import asyncio

async def wrapped():
    print(5)
    return 'result-wrapped'

async def inner(task):
    print(4)
    print('inner', task)
    result = await task
    print('task return ', result)
    
async def starter():
    print(2)
    task = asyncio.ensure_future(wrapped())
    print(3)
    await inner(task)
    print(6)
    

event_loop = asyncio.get_event_loop()
try:
    print(1)
    event_loop.run_until_complete(starter())
finally:
    event_loop.close()

1
2
3
4
inner <Task pending coro=<wrapped() running at <ipython-input-1-e0a7a67b0c22>:3>>
5
task return  result-wrapped
6


## Waiting for Multiple Coroutines

In [1]:
import asyncio

async def phase(i):
    print('in phase', i)
    await asyncio.sleep(0.1*i)
    print('done with phase', i)
    return f"phase result {i}"

async def main(num):
    print('main start')
    phases = [phase(i) for i in range(num)]
    print('wait phases complete')
    completed, pending = await asyncio.wait(phases)
    results = [t.result() for t in completed]
    print(results,1)
    
event_loop = asyncio.get_event_loop()
try:
    print(1)
    event_loop.run_until_complete(main(3))
finally:
    event_loop.close()

1
main start
wait phases complete
in phase 0
in phase 2
in phase 1
done with phase 0
done with phase 1
done with phase 2
['phase result 2', 'phase result 0', 'phase result 1'] 1


In [1]:
# wait timeout
import asyncio

async def phase(i):
    print('in phase', i)
    try:
        await asyncio.sleep(0.1*i)
    except asyncio.CancelledError:
        print('phase cancel', i)
        raise
    else:
        print('done with phase', i)
        return f"phase result {i}"

async def main(num):
    print('main start')
    phases = [phase(i) for i in range(num)]
    print('wait phases complete')
    completed, pending = await asyncio.wait(phases, timeout=0.1)
    print(f'{len(completed)} completed, {len(pending)} pending')
    if pending:
        print('to cancel tasks')
        for t in pending:
            t.cancel()
    print('exiting main')
    
event_loop = asyncio.get_event_loop()
try:
    print(1)
    event_loop.run_until_complete(main(3))
finally:
    event_loop.close()

1
main start
wait phases complete
in phase 0
in phase 2
in phase 1
done with phase 0
1 completed, 2 pending
to cancel tasks
exiting main
phase cancel 1
phase cancel 2


## Gathering Results from Coroutines

In [1]:
import asyncio

async def phase1():
    print('in phase1')
    await asyncio.sleep(3)
    print('done with phase1')
    return f"phase result 1"

async def phase2():
    print('in phase2')
    await asyncio.sleep(1)
    print('done with phase2')
    return f"phase result 2"

async def main():
    result = await asyncio.gather(phase1(), phase2())
    print(result)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main())
finally:
    event_loop.close()

in phase2
in phase1
done with phase2
done with phase1
['phase result 1', 'phase result 2']


## Handling Background Operations as They Finish

In [1]:
import asyncio

async def phase(i):
    print('in phase', i)
    await asyncio.sleep((1-0.1*i))
    print('done with phase', i)
    return f"phase result {i}"

async def main(num):
    print('main start')
    phases = [phase(i) for i in range(num)]
    print('wait phases complete')
    results = []
    for next_to_complete in asyncio.as_completed(phases):
        answer = await next_to_complete
        print("received ", answer)
        results.append(answer)
    print(results,1)
    return results
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(3))
finally:
    event_loop.close()

main start
wait phases complete
in phase 1
in phase 0
in phase 2
done with phase 2
received  phase result 2
done with phase 1
received  phase result 1
done with phase 0
received  phase result 0
['phase result 2', 'phase result 1', 'phase result 0'] 1


## Locks

In [1]:
import asyncio
import functools

def unlock(lock):
    print("callback release the lock")
    lock.release()
    
async def coro1(lock):
    print('coro1 wait the lock')
    with await lock:
        print('coro1 acquire lock')
    print('coro1 release lock')
    
async def coro2(lock):
    print('coro2 wait the lock')
    await lock
    try:
        print('coro2 acquire lock')
    finally:
        print('coro2 release lock')
        lock.release()
        
async def main(loop):
    lock = asyncio.Lock()
    await lock.acquire()
    print('lock acquired', lock.locked())
    
    # schedule a callback to unlock the lock
    loop.call_later(2, functools.partial(unlock, lock))
    
    await asyncio.wait([coro1(lock), coro2(lock)])
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

lock acquired True
coro1 wait the lock
coro2 wait the lock
callback release the lock
coro1 acquire lock
coro1 release lock
coro2 acquire lock
coro2 release lock


## Events

In [1]:
import asyncio
import functools

def set_event(event):
    print("set event in callback ")
    event.set()
    
async def coro1(event):
    print('coro1 wait the event')
    await event.wait()
    print('coro1 triggled')
    
async def coro2(event):
    print('coro2 wait the event')
    await event.wait()
    print('coro2 triggled')
        
async def main(loop):
    event = asyncio.Event()
    print('event state a', event.is_set())
    
    loop.call_later(2, functools.partial(set_event, event))
    
    await asyncio.wait([coro1(event), coro2(event)])
    print('event state b', event.is_set())
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

event state a False
coro1 wait the event
coro2 wait the event
set event in callback 
coro1 triggled
coro2 triggled
event state b True


## Conditions

In [1]:
import asyncio

async def consumer(condition, n):
    with await condition:
        print(f'consumer {n} is waiting')
        await condition.wait()
        print(f'consumer {n} trigger')
    print(f'consumer {n} end')
    
async def manipulate_condition(condition):
    print('Starting manipulate')
    
    # wait for consumer start
    await asyncio.sleep(0.1)
    
    for i in range(1,3):
        with await condition:
            print(f'notifying {i} consumers')
            condition.notify(n=i)
        await asyncio.sleep(2)
    with await condition:
        print('notifying remain consumers')
        condition.notify_all()
        
async def main(loop):
    condition = asyncio.Condition()
    consumers = [consumer(condition, i) for i in range(5)]
    loop.create_task(manipulate_condition(condition))
    await asyncio.wait(consumers)
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop))
finally:
    event_loop.close()

Starting manipulate
consumer 2 is waiting
consumer 1 is waiting
consumer 3 is waiting
consumer 4 is waiting
consumer 0 is waiting
notifying 1 consumers
consumer 2 trigger
consumer 2 end
notifying 2 consumers
consumer 1 trigger
consumer 1 end
consumer 3 trigger
consumer 3 end
notifying remain consumers
consumer 4 trigger
consumer 4 end
consumer 0 trigger
consumer 0 end


## Queues

In [1]:
import asyncio

async def consumer(n, q):
    print(f"consumer {n} starting")
    while True:
        print(f"consumer {n} wait for item")
        item = await q.get()
        print(f"consumer {n} get {item}")
        if item is None:
            # None is the signal to stop.
            q.task_done()
            break
        else:
            await asyncio.sleep(0.01*item)
            q.task_done()
    print(f"consumer {n} end")
    
async def producer(q, num_workers):
    print("producer starting")
    # Add some numbers to the queue to simulate jobs
    for i in range(num_workers *3):
        await q.put(i)
        print(f"producer add task {i} to queue")
    
    print("producer add stop signal to queue")
    for i in range(num_workers):
        await q.put(None)
    print("producer wait for queue to empty")  
    await q.join()
    print("producer ending")
    
async def main(loop, num_consumers):
    q = asyncio.Queue(maxsize=num_consumers)
    consumers = [loop.create_task(consumer(i, q)) for i in range(num_consumers)]
    prod = loop.create_task(producer(q, num_consumers))
    await asyncio.wait(consumers + [prod])
    
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main(event_loop, 2))
finally:
    event_loop.close()

consumer 0 starting
consumer 0 wait for item
consumer 1 starting
consumer 1 wait for item
producer starting
producer add task 0 to queue
producer add task 1 to queue
consumer 0 get 0
consumer 1 get 1
producer add task 2 to queue
producer add task 3 to queue
consumer 0 wait for item
consumer 0 get 2
producer add task 4 to queue
consumer 1 wait for item
consumer 1 get 3
producer add task 5 to queue
producer add stop signal to queue
consumer 0 wait for item
consumer 0 get 4
consumer 1 wait for item
consumer 1 get 5
producer wait for queue to empty
consumer 0 wait for item
consumer 0 get None
consumer 0 end
consumer 1 wait for item
consumer 1 get None
consumer 1 end
producer ending
